In [59]:
import boto3
import numpy as np
import pprint as pp
import os

In [60]:
# feed AWS credentials to boto3
AWS_ACCESS_KEY = '' 
AWS_SECRET_KEY = ''
REGION = '' #ex: 'us-east-1'

# connect to aws console using boto3
session = boto3.Session(aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_KEY,
                        region_name=REGION)
#connect to the the amazon web service "rekognition" using boto3
client = session.client('rekognition')

# now, the path of the folder that contains the pictures to be sorted
PICTURE_FOLDER = r''

In [61]:
# BELOW ARE THE FUNCTIONS

"""
This function creates 8 folders, if they don't already exist in the current directory. 
These folders are named for the age-ranges they represent. This is a helper function.
"""
def create_folders():
    # if folders don't exist..
    name_list = []
    age_ranges = {'kids':'Ages 0-12', 'teenagers':'Ages 13-18','young adults': 'Ages 18-25',
                 'mid adults':'Ages 25-35', 'adults':'Ages 35-45', 'middle aged':'Ages 45-55',
                 'older adults':'Ages 55-70', 'elderly':'Ages 70-110'}
    #kids
    name_list.append(os.getcwd() + '\\' + age_ranges['kids'])
    #teenagers
    name_list.append(os.getcwd() + '\\' +  age_ranges['teenagers'])
    #young adults
    name_list.append(os.getcwd() + '\\' +  age_ranges['young adults'])
    #mid adults
    name_list.append(os.getcwd() + '\\' +  age_ranges['mid adults'])
    #adults 
    name_list.append(os.getcwd() + '\\' +  age_ranges['adults'])
    #middle aged
    name_list.append(os.getcwd() + '\\' +  age_ranges['middle aged'])
    # older adults
    name_list.append(os.getcwd() + '\\' +  age_ranges['older adults'])
    # elderly
    name_list.append(os.getcwd() +  '\\' + age_ranges['elderly'])
    for item in name_list:
        #if folder exists in dir, continue
        if item.split('\\')[-1] in os.listdir():
            continue
        try:
            os.mkdir(item)
        except OSError:
            print ("Creation of the directory %s failed" % item)
        else:
            print ("Successfully created the directory %s " % item)
            
"""
input: object returned from 'session.client('rekognition')'
output: average age for all the faces in a single .jpg
"""
def get_avg_age(client, PATH='input.jpg'):
    client = session.client('rekognition')
    with open(PATH,'rb') as image:
        response = client.detect_faces(Image={'Bytes':image.read(),},Attributes=['ALL'])
    ages = []
    for face in response['FaceDetails']:
        ages.append(face['AgeRange']['Low'])
        ages.append(face['AgeRange']['High'])
    return [PATH,int(np.average(ages))]

"""
input: none
output: a list that looks like this:
        # ['path of picture', 'avg_age of faces in picture'] 
        # [str, int]
"""
def tag_pics():
    
    tagged_pics = []
    # for every picture in the folder...
    for filename in os.listdir(PICTURE_FOLDER):
        # ignore .ipynb_checkpoints
        # this is just something that occurs rn working within jupyter notebooks
        if filename == '.ipynb_checkpoints':
            continue
        path = PICTURE_FOLDER + filename 
        # open picture
        with open(path,'rb') as image:
            # call avg_age for each photo in folder and append to list
            tagged_pics.append( get_avg_age(client,PATH=path) )
        #pp.pprint(tagged_pics)
        # return a list with items that look like this:
        # ['path of picture', 'avg_age of faces in picture'] 
        # [str, int]
    return tagged_pics

"""
This function takes the list returned by the tag_pics function and creates folders based
on the average age of the faces in each photo, as predicted by AWS rekognition

input: the list returned by tag_pics()
output: none
"""
def sort_pics(tagged_pics):
    #create folders if they don't exist
    create_folders()
    #place pics in folders
    for item in tagged_pics:
        # item[0] is path
        # item[1] is int
        current_path = item[0]
        age = item[1]
        if age > 0 and age <= 12:
            new_path = os.getcwd() + '\\Ages 0-12\\' + str(age) + '-' + item[0].split('\\')[-1] 
            os.rename(current_path, new_path)
        elif age > 12 and age <= 18:
            new_path = os.getcwd() + '\\Ages 12-18\\' + str(age) + '-' +  item[0].split('\\')[-1] 
            os.rename(current_path, new_path)
        elif age > 18 and age <= 25:
            new_path = os.getcwd() + '\\Ages 18-25\\' + str(age) + '-' + item[0].split('\\')[-1]
            os.rename(current_path, new_path)
        elif age > 25 and age <= 35:
            new_path = os.getcwd() + '\\Ages 25-35\\' + str(age) + '-' +  item[0].split('\\')[-1] 
            os.rename(current_path, new_path)
        elif age > 35 and age <= 45:
            new_path = os.getcwd() + '\\Ages 35-45\\' + str(age) + '-' +  item[0].split('\\')[-1] 
            os.rename(current_path, new_path)
        elif age > 45 and age <= 55:
            new_path = os.getcwd() + '\\Ages 45-55\\' + str(age) + '-' + item[0].split('\\')[-1]
            os.rename(current_path, new_path)
        elif age > 55 and age <= 70:
            new_path = os.getcwd() + '\\Ages 55-70\\' + str(age) + '-' +  item[0].split('\\')[-1]
            os.rename(current_path, new_path)
        elif age > 70 and age <= 110:
            new_path = os.getcwd() + '\\Ages 70-110\\' + str(age) + '-' +  item[0].split('\\')[-1]
            os.rename(current_path, new_path)
        else:
            return "AGE_VARIABLE_ERROR"
    



In [62]:
# Testing!
tagged_pics = tag_pics()
sort_pics(tagged_pics)

Successfully created the directory C:\Users\Nick\0_sort_faces_by_age\Ages 0-12 
Successfully created the directory C:\Users\Nick\0_sort_faces_by_age\Ages 13-18 
Successfully created the directory C:\Users\Nick\0_sort_faces_by_age\Ages 18-25 
Successfully created the directory C:\Users\Nick\0_sort_faces_by_age\Ages 25-35 
Successfully created the directory C:\Users\Nick\0_sort_faces_by_age\Ages 35-45 
Successfully created the directory C:\Users\Nick\0_sort_faces_by_age\Ages 45-55 
Successfully created the directory C:\Users\Nick\0_sort_faces_by_age\Ages 55-70 
Successfully created the directory C:\Users\Nick\0_sort_faces_by_age\Ages 70-110 
